In [1]:
import numpy as np      
import trimesh           
import pyglet
import scipy 
import pandas as pd
#In the following cells we first define several functions needed for the (interactive) analysis of neck diameters & membrane distance

In [2]:
# a function to read a .tsi file and return the vertices, triangles and inclusions
def read_sections(file_path):
    section_delimiter = "     "  # Delimiter of five spaces
    
    # Initialize lists for each section
    section1 = []
    section2 = []
    section3 = []
    
    # Flag to indicate the current section being read
    current_section = None
    
    # Read the file
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            
            # Identify the section based on the starting word
            if line.startswith("vertex"):
                current_section = section1
                print(line)
                continue
            elif line.startswith("triangle"):
                current_section = section2
                print(line)
                continue
            elif line.startswith("inclusion"):
                current_section = section3
                print(line)
                continue
            
            # Skip the lines that mark the beginning of each section
            if current_section is None:
                continue
            
            # Split the line using the section delimiter
            if section_delimiter in line:
                entries = line.split(section_delimiter)
                # Remove empty entries (caused by consecutive delimiters)
                entries = [entry.strip() for entry in entries if entry.strip()]
                # Discard the first entry of each section
                if entries:
                    entries.pop(0)
                    # Convert entries based on the current section
                    if current_section is section1:
                        entries = [float(entry) for entry in entries]
                    elif current_section is section2:
                        entries = [int(entry) for entry in entries]
                    elif current_section is section3:
                        entries = [int(entry) if idx < len(entries) - 2 else float(entry) for idx, entry in enumerate(entries)]

                current_section.append(entries)
    
    return section1, section2, section3

In [3]:
#read the size of the simulation box from the input .tsi file
def read_box(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()        
        second_line = lines[1].strip().split()
        # Check if the second line starts with 'box'
        if second_line[0] != 'box':
            raise ValueError("The second line does not start with 'box'.")
        # Convert the remaining elements to floats
        box = [float(value) for value in second_line[1:]]
        return box

In [6]:
#make the mesh in trimesh, show a visualization if prompted
def make_mesh(file_path,visualize): #visualize: 'yes'/'no'
    vertex,triangle,inclusion = read_sections(file_path) #reads in and prints header of each section
    mesh = trimesh.Trimesh(vertices=vertex,faces=triangle)
    Newmesh=mesh
    if visualize=='yes':
        Newmesh.show()
    return Newmesh

In [8]:
#if one wants to compare surface area of inner and outer mesh
def inner_outer_area(new_mesh_inner, new_mesh_outer):#area of inner/outer surface
    area_in,area_out=new_mesh_inner.area, new_mesh_outer.area #can also use np.sum(mesh.area_faces), same result
    print('Area inner surface:',area_in,'Area outer surface:',area_out)
    return area_in,area_out

In [10]:
#for full vesicles: detect edges of the open mesh (inner or outer), group them into necks
def poregroupsNoPBC(new_mesh_outer):
    unique_edges = new_mesh_outer.edges[trimesh.grouping.group_rows(new_mesh_outer.edges_sorted, require_count=1)]
    edges=unique_edges
    groups = []  #empty list to store groups
    while len(edges) > 0:
        j = edges[0]  # Set j to the first edge of unvisited(=not yet removed) boundary edges
        g = [j]  # add the first edge to the current group
        edges = np.delete(edges, 0, axis=0)  # Remove the first edge from the array of unvisited boundary edges

        while True:
            found = False
            for i in edges:
                if j[0] == i[0] or j[0] == i[1] or j[1] == i[0] or j[1] == i[1]: #if edges touch
                    g.append(i)  # Append the current edge to the current group
                    edges = np.delete(edges, np.where(np.all(edges == i, axis=1)), axis=0)  # Remove from unvisited edges
                    j = i  # Set j to the current edge
                    found = True
                    break  # Begin again with the new j
        
            if not found:
              #  print('all edges checked, no follower found')
                groups.append(g)  # Append the current group to groups
                break  # Exit the inner while loop to start a new group with the next remaining edge
    print('Number of detected pore groups:',len(groups))
    return groups

In [11]:
#calculate the position/ coordinate (center of mass) of a pore group, i.e. a neck
def poreposition(groups,mesh): 
    vertex_groups=[] #make the groups of edges(noPBC) into groups of vertices
    for i in groups:
        i=np.array(i)
        vertex_groups.append(i.flatten())
        
    pore_positions=[]
    for i in vertex_groups:#go through the groups of vertices, average their position
        group_positions=mesh.vertices[i]
        position=np.sum(group_positions,axis=0)/len(i)
        pore_positions.append(position)
            
    pore_positions=np.array(pore_positions)
    print('Pore positions:',pore_positions)
    return pore_positions

In [12]:
#calculate neck diameter from pore groups
def poresizeNoPBC(genus,groups,new_mesh_outer): #genus= number of necks
    group_diameters=[]
    for i in groups:
        circumference=0
        for j in i:
            j=np.array(j) #one edge of the group
            v1_index,v2_index=j[0],j[1]
            v1,v2=new_mesh_outer.vertices[v1_index],new_mesh_outer.vertices[v2_index]
            leng=np.linalg.norm(v1-v2)
            circumference=circumference+leng
            
        group_diameter=circumference/np.pi
        group_diameters.append(group_diameter)

    if genus==len(groups): 
        pore_sizes=group_diameters
    elif genus<len(groups):#we know that there are groups fragmented
        print('Too many necks found -> pore size calc. may be off! Check results, adjust threshold for inner-outer-mesh division.')
        pore_sizes=group_diameters
    else:
        print('Some necks have not been detected or were falsely merged -> pore size calc. may be off! Check results, adjust threshold for inner-outer-mesh division.')
        pore_sizes=group_diameters
    print('Pore diameters:',pore_sizes)
    return pore_sizes

In [ ]:
#calculate average radius of a membrane/mesh, by the vertices average distance to the center of the box; box center has to be user-adjusted if different boxsize is used or if the mesh is not centered in the box!
def mean_vertex_distance(mesh):
    # Get the center of mass of the mesh
    #center_of_mass = mesh.center_mass

    # Get the coordinates of the vertices
    vertices = mesh.vertices
    
    # Calculate the distance from each vertex to the center of mass or box center
    #distances = np.linalg.norm(vertices - center_of_mass, axis=1)
    distances = np.linalg.norm(vertices - np.array([50.0,50.0,50.0]), axis=1) #box center (50,50,50) has to be user-adjusted if different boxsize is used!
    
    # Return the mean distance
    return np.mean(distances)

In [992]:
#divide the mesh into inner and outer membrane based on a threshold value for the distance of a vertex to the convex hull of the mesh; this threshold has to be user-adapted at each analysed frame to detect all necks
def inner_outer2(Newmesh, visualize,threshold): #threshold should be ca. between 2.0 and 0.7 (for small core sizes, i.e. large mem-distance vs. large core sizes, i.e. small mem-distance)
    triangles, vertices = Newmesh.triangles, Newmesh.vertices
    convex_hull = mesh.convex_hull
    inner,outer_hull= [],[]# Initialize list for indices of inner triangles/triangles in pores and outer ones
    for i in range(len(triangles)):
        for j in triangles[i]: #go through the vertex positions
            closest_point, dist, hit_id = trimesh.proximity.closest_point(convex_hull,[j]) #find the closest point to the current vertex on the convex hull, and the distance between both
            if dist>threshold:
                inner.append(i)
        if i not in inner:
            outer_hull.append(i)
            

    outer_mesh=trimesh.Trimesh(vertices=Newmesh.vertices, faces=Newmesh.faces[outer_hull])
    inner_mesh=trimesh.Trimesh(vertices=Newmesh.vertices, faces=Newmesh.faces[inner])

    if visualize=='yes':
        print('outer hull is shown:')
        outer_mesh.show()
    return inner_mesh, outer_mesh

In [1434]:
#active analysis: read in file, make mesh and divide into inner & outer mesh
file='./ExpansionFiles/K10Equil-Pro/TSI506/dts1450.tsi'
boxsize=read_box(file)
mesh=make_mesh(file, 'noPBC',boxsize,'no')
inner_mesh,outer_mesh=inner_outer2(mesh,'no')

vertex                1800
triangle                3676
inclusion                 180


In [1435]:
#check inner mesh visually for good neck detection/ if unsure whether threshold was chosen well
#inner_mesh.show()

In [ ]:
#find groups of vertices lining each neck, their positions, the diameters and print the mean and std of the 21 necks
groups=poregroupsNoPBC(outer_mesh)
pore_positions=poreposition(groups,mesh)
pore_sizes=poresizeNoPBC(20,groups,outer_mesh)
print(np.mean(pore_sizes),np.std(pore_sizes))

In [ ]:
#find average radii of inner and outer mesh and save them (add them as the last 2 entries to the array of neck diameters, which is called "pore_sizes")
#(to later calculate/plot the average membrane distance)
mean_distance1 = mean_vertex_distance(inner_mesh)
print(f'Mean distance inner mesh from center of box: {mean_distance1}')
mean_distance2 = mean_vertex_distance(outer_mesh)
print(f'Mean distance outer mesh from center of box: {mean_distance2}')
pore_sizes.append(mean_distance1)
pore_sizes.append(mean_distance2)

In [1442]:
diameters_to_save=np.array(pore_sizes) #make the list of neck diameters & membrane distance into an array
print(np.mean(diameters_to_save[:-2])) #check the average neck diameter one last time
np.save('./ExpansionFiles/ResultsK10-EquilPro/Porediameters-K10Pro-pos3-file50.npy', diameters_to_save) #save as a .npy file for later analysis/plots